In [1]:
## For google colab
from google.colab import drive
drive.mount('/content/drive')
!pip install geopandas &> /dev/null

data_path = '/content/drive/MyDrive/shared/ssc22-case-comp/dataset/'
module_path = '/content/drive/MyDrive/shared/ssc22-case-comp/sonny_dir/custom_modules/'
out_path = '/content/drive/MyDrive/shared/ssc22-case-comp/dataset/'

Mounted at /content/drive


In [ ]:
## For compCan
data_path = '/home/projects/joosungm/def-lelliott/joosungm/projects/ssc-comp/'
module_path = '/home/projects/joosungm/def-lelliott/joosungm/projects/ssc-comp/'
out_path = '/home/projects/joosungm/def-lelliott/joosungm/projects/ssc-comp/'

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import pickle
import sys
from tqdm.auto import tqdm

os.chdir(data_path)
print(os.getcwd())

sys.path.append(os.path.abspath(module_path))
import weighted_average as WA ## Load a custom module

/content/drive/MyDrive/shared/ssc22-case-comp/dataset


In [ ]:
## Load canada data
can_path = './python_canada-speed-data.p'
with open('./python_canada-speed-data.p', 'rb') as file:
    can_dat = pickle.load(file)

can_dat['centroid'] = can_dat['geometry'].centroid

print('data load complete')

data load complete


In [ ]:
## Parallel processing ##

# test_dat = can_dat.loc[0:100, :]
# test_dat.head(5)
# pddf = pd.DataFrame(test_dat)

pddf = pd.DataFrame(can_dat)

unique_qk = gpd.GeoDataFrame(pddf.drop_duplicates(subset=['quadkey'], keep='first')) # make sure the data type is GeoDataFrame.
unique_dest = WA.get_dest_df(unique_qk)

import multiprocessing as mp
splits = np.array_split(unique_qk, mp.cpu_count())

def parallel_dist(split_dat):
    global unique_dest
    split_dat['distance'] = split_dat['centroid'].apply(lambda x: np.min(unique_dest.distance(x))/1000)
    return split_dat

pool = mp.Pool(mp.cpu_count())

results = pool.map(parallel_dist, [split_dat for split_dat in splits])

pool.close()   

dist_df = pd.concat(results)

with open(out_path+'dist-to-pc-overall.p', 'wb') as file:
    pickle.dump(dist_df, file)
dist_df.to_csv(out_path+'dist-to-pc-overall.csv', index=False)

## Merge the distance file to the original data.
can_merged = can_dat.merge(dist_df, on='quadkey', how='left')
with open(out_path+'can-speed-tiles-with-dist.p', 'wb') as file:
    pickle.dump(can_merged, file)
print('job done!')

In [ ]:
## Compute the weighted averages

with open('./can-speed-tiles-with-dist.p', 'rb') as file:
    new_dat = pickle.load(file)
print(new_dat.shape)
new_dat.head(5)

In [8]:
! pip install --upgrade pandas

In [10]:
! python3 --version

Python 3.7.13
